In [1]:
import sys
sys.path.append('..')
import importlib
import neural_policies
import main
import game_utils
import policy_gradient

import torch
import yaml
import numpy as np
import pyspiel
from types import SimpleNamespace

importlib.reload(main)
importlib.reload(neural_policies)
importlib.reload(game_utils)
importlib.reload(policy_gradient)

seed = 1
torch.random.manual_seed(seed)
np.random.seed(seed)

In [2]:
# game = game_utils.create_random_normal_game(5)
# game = main.create_game(SimpleNamespace(game="rps"))
game = main.create_game(SimpleNamespace(game="kuhn_poker"))
# nn_config_file = "../config/linear_mlp.yaml"
# nn_config_file = "../config/kuhn_mlp.yaml"
nn_config_file = "../config/kuhn_big.yaml"
reinforce_config_file = "../config/reinforce_train.yaml"
nn_config = yaml.safe_load(open(nn_config_file, "r"))
reinforce_config = yaml.safe_load(open(reinforce_config_file, "r"))

In [3]:
nash_policy, nash_value = main.nash_equilibrium_policy_and_value(game)
print("nash value", nash_value)
nn_player = 0
nn_policy = neural_policies.create_policy_net(game, nn_config)
nn_tab_policy = neural_policies.nn_to_tabular_policy(game, nn_policy, nn_player)

Exploitability: 0.06869879381715754
nash value [-0.05311271  0.05311271]


# ignore from here

In [4]:
# s = game_utils.all_game_states(game, 0)[0]
# s.legal_actions_mask()
# s.information_state_tensor()

In [5]:
from policy_gradient import PolicyGradientHypernetTrainer

# hypernet = main.train_hypernet_actionoutput(game, nn_config, nn_player, nn_policy, "../config/hypernet.yaml", "../config/hypernet_reinforce_train.yaml")

hypernet_config = "../config/hypernet.yaml"
hypernet_config = yaml.safe_load(open(hypernet_config, "r"))
hypernet_train_config = "../config/hypernet_reinforce_train.yaml"
hypernet_train_config = yaml.safe_load(open(hypernet_train_config, "r"))

mlp = neural_policies.create_policy_net(game, nn_config)
# hypernet = neural_policies.create_hypernet_nn_output(game, mlp, hypernet_config)
hypernet = neural_policies.create_hypernet_actionoutput(game, mlp, hypernet_config)
trainer = PolicyGradientHypernetTrainer(game, hypernet, hypernet_train_config)

  0%|          | 0/100000 [00:00<?, ?it/s]


AttributeError: 'HyperNetworkActionOutput' object has no attribute 'model_weight_output'

In [6]:
trainer.train_best_response(nn_config, 1 - nn_player)

  0%|          | 217/100000 [02:48<25:32:10,  1.09it/s]

[[0.67881411 0.32118592]
 [0.01395872 0.98604125]
 [0.64661425 0.35338572]
 [0.38737103 0.61262906]
 [0.763515   0.23648497]
 [0.68917471 0.31082529]
 [0.5        0.5       ]
 [0.5        0.5       ]
 [0.5        0.5       ]
 [0.5        0.5       ]
 [0.5        0.5       ]
 [0.5        0.5       ]]
[[0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.  1. ]
 [0.  1. ]
 [0.  1. ]
 [0.  1. ]
 [0.  1. ]
 [1.  0. ]]
value of table br policy [-0.70484179  0.70484179]
[[5.00000000e-01 5.00000000e-01]
 [5.00000000e-01 5.00000000e-01]
 [5.00000000e-01 5.00000000e-01]
 [5.00000000e-01 5.00000000e-01]
 [5.00000000e-01 5.00000000e-01]
 [5.00000000e-01 5.00000000e-01]
 [7.79365229e-24 1.00000000e+00]
 [2.26749266e-25 1.00000000e+00]
 [3.91736507e-01 6.08263493e-01]
 [6.25857541e-25 1.00000000e+00]
 [5.27535791e-25 1.00000000e+00]
 [2.37434585e-25 1.00000000e+00]]
value of hypernet br policy [-0.53696586  0.53696586]
(0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0) -10.0 -6

  0%|          | 313/100000 [05:14<27:52:03,  1.01s/it] 


KeyboardInterrupt: 

In [6]:
nn_policy = neural_policies.create_policy_net(game, nn_config)
nn_tab_policy = neural_policies.nn_to_tabular_policy(game, nn_policy, nn_player)
main.tabular_br(game, nn_player, nn_tab_policy)
main.policy_gradient_br(
    game, nn_config, reinforce_config, nn_player, nn_policy, nn_tab_policy
)
main.hypernet_br(game, hypernet, nn_player, nn_policy, nn_tab_policy)

lst_policy = None

nn policy with tabular br value [-0.1596228  0.1596228]
value of nn br policy [-0.15929753  0.15929753]
value of hypernet br policy [-0.04876066  0.04876066]


In [7]:
# print(game.row_utilities())

new_input_nn = neural_policies.create_policy_net(game, nn_config)
input_probs = neural_policies.nn_to_tabular_policy(game, new_input_nn, nn_player).action_probability_array
# input_probs = neural_policies.get_nn_probs(new_input_nn, game.new_initial_state(), nn_player)
# print("input probs", input_probs)

playable_states = game_utils.all_game_states(game, 1-nn_player)
state = playable_states[0]

# print(neural_policies.get_hypernet_probs(hypernet, new_input_nn, state, 1-nn_player))
# print(hypernet(new_input_nn, torch.tensor(state.information_state_tensor(1-nn_player))))
nn_response = neural_policies.nn_to_tabular_policy(game, hypernet, 1-nn_player, new_input_nn)
# print("nn response", nn_response.action_probability_array)
br = main.compute_best_response_tabular_policy(game, nn_tab_policy, 1 - nn_player)
print(lst_policy)
if lst_policy is not None:
    cur_policy = br.action_probability_array
    # find rows that differ
    diff = np.where(cur_policy != lst_policy)
    print("diff", diff)
lst_policy = br.action_probability_array
print(br.action_probability_array)

# neural_br_policy = neural_policies.create_policy_net(game, nn_config)
# trainer = policy_gradient.PolicyGradientTrainer(
#     game, neural_br_policy, train_params=reinforce_config
# )
# trainer.train_best_response(nn_policy, 1-nn_player)
# print(neural_br_policy(torch.tensor(game.new_initial_state().information_state_tensor(1-nn_player))))
# print(neural_policies.get_nn_probs(neural_br_policy, game.new_initial_state(), 1-nn_player))

None
[[0.33333333 0.33333333 0.33333333]
 [1.         0.         0.        ]]


In [ ]:
from pprint import pprint
# pprint(list(hypernet._model.get_weights()))
mm = hypernet._model
[l.bias for l in mm.layers]
# mm.layers[-1].weight

In [7]:
import input_net_data
importlib.reload(input_net_data)
# input_nets = input_net_data.read_all_nn(game, "../nn_inputs/kuhn")
input_nets = input_net_data.read_all_nn(game, "../nn_inputs/kuhn-0-big")
# input_nets = input_net_data.read_all_nn(game, "../nn_inputs/rps")

In [8]:
ind = -1
nn_player

0

In [51]:
# nn_policy = neural_policies.create_policy_net(game, nn_config)
ind = ind + 1
# import random
# ind = random.choice([i for i in range(5)])
ind = ind % 5
print("ind", ind)
# print(input_nets[ind].get_weights())
nn_tab_policy = neural_policies.nn_to_tabular_policy(game, input_nets[ind], 0)
# br_pol = main.tabular_br(game, nn_player, nn_tab_policy)
print(nn_tab_policy.action_probability_array)

# print(nn_tab_policy.action_probability_array)

br = main.compute_best_response_tabular_policy(game, nn_tab_policy, 1)
print(br.action_probability_array)
combined_policy = main.combine_tabular_policies(game, nn_tab_policy, br)
print("value of table br policy", main.policy_value(game, combined_policy))

nn_br = neural_policies.nn_to_tabular_policy(game, hypernet, 1-nn_player, input_nets[ind])
print(nn_br.action_probability_array)
diff = np.abs(br.action_probability_array - nn_br.action_probability_array).sum()
print(diff)
combined_policy = main.combine_tabular_policies(game, nn_tab_policy, nn_br)
print("value of table br policy", main.policy_value(game, combined_policy))

ind 4
[[0.8914901  0.10850996]
 [0.06324194 0.9367581 ]
 [0.00770295 0.99229699]
 [0.00207829 0.99792176]
 [0.94578105 0.05421892]
 [0.31916466 0.68083537]
 [0.5        0.5       ]
 [0.5        0.5       ]
 [0.5        0.5       ]
 [0.5        0.5       ]
 [0.5        0.5       ]
 [0.5        0.5       ]]
[[0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.  1. ]
 [0.  1. ]
 [0.  1. ]
 [0.  1. ]
 [1.  0. ]
 [1.  0. ]]
value of table br policy [-0.46546748  0.46546748]
[[0.5        0.5       ]
 [0.5        0.5       ]
 [0.5        0.5       ]
 [0.5        0.5       ]
 [0.5        0.5       ]
 [0.5        0.5       ]
 [0.96017069 0.03982927]
 [0.90566278 0.09433722]
 [0.03334772 0.96665221]
 [0.33968946 0.66031045]
 [0.82986069 0.1701393 ]
 [0.74316478 0.25683516]]
5.331690486520529
value of table br policy [-0.0595021  0.0595021]


In [17]:
for s in combined_policy.states:
    print(s.information_state_tensor(), s.information_state_string())

[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0
[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0] 0pb
[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 1
[1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0] 1pb
[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 2
[1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0] 2pb
[0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0] 1p
[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0] 1b
[0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0] 2p
[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0] 2b
[0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0p
[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0] 0b


In [ ]:
from nfn.common import state_dict_to_tensors
state_dict_to_tensors(input_nets[0].state_dict())[0][0].shape

In [ ]:
nn = input_nets[0]
w1 = list(nn.parameters())[0]

# relevant from here, next two cells

In [4]:
import policy_gradient
importlib.reload(policy_gradient)
importlib.reload(neural_policies)

hypernet_config = "../config/hypernet.yaml"
hypernet_config = yaml.safe_load(open(hypernet_config, "r"))
hypernet_train_config = "../config/hypernet_reinforce_train.yaml"
hypernet_train_config = yaml.safe_load(open(hypernet_train_config, "r"))

mlp = neural_policies.create_policy_net(game, nn_config)
hypernet_nn_output = neural_policies.create_hypernet_nn_output(game, mlp, hypernet_config)
trainer = policy_gradient.PolicyGradientHypernetTrainer(game, hypernet_nn_output, hypernet_train_config)

In [5]:
policy_nets = trainer.train_simultaneous_br(nn_config)
# trainer.train_best_response(policy_nets, 0)

  0%|          | 0/100000 [00:00<?, ?it/s]

[[0.33333333 0.33333333 0.33333333]
 [0.26185703 0.46273056 0.27541241]]
[[0.         0.         1.        ]
 [0.33333333 0.33333333 0.33333333]]
value of table br policy [ 0.20087352 -0.20087352]
[[0.42134953 0.12136761 0.4572829 ]
 [0.33333333 0.33333333 0.33333333]]
value of hypernet br policy [ 0.01128443 -0.01128443]
[[0.35675135 0.27289236 0.37035629]
 [0.33333333 0.33333333 0.33333333]]
[[0.33333333 0.33333333 0.33333333]
 [1.         0.         0.        ]]
value of table br policy [-0.09746394  0.09746394]
[[0.33333333 0.33333333 0.33333333]
 [0.37019143 0.12950855 0.50029999]]
value of hypernet br policy [ 0.0076363 -0.0076363]
[[0.33333333 0.33333333 0.33333333]
 [0.26185703 0.46273056 0.27541241]]
[[0.         0.         1.        ]
 [0.33333333 0.33333333 0.33333333]]
value of table br policy [ 0.20087352 -0.20087352]
[[0.40974024 0.12008872 0.47017103]
 [0.33333333 0.33333333 0.33333333]]
value of hypernet br policy [ 0.01606528 -0.01606528]
[[0.35675135 0.27289236 0.3703

  0%|          | 261/100000 [00:06<39:18, 42.29it/s]  

[[3.33333333e-01 3.33333333e-01 3.33333333e-01]
 [6.78038239e-01 4.40306234e-04 3.21521401e-01]]
[[0.         1.         0.        ]
 [0.33333333 0.33333333 0.33333333]]
value of table br policy [ 0.35651684 -0.35651684]
[[0.31381807 0.01667857 0.66950339]
 [0.33333333 0.33333333 0.33333333]]
value of hypernet br policy [-0.34694687  0.34694687]
[[0.31720075 0.01668318 0.66611606]
 [0.33333333 0.33333333 0.33333333]]
[[0.33333333 0.33333333 0.33333333]
 [1.         0.         0.        ]]
value of table br policy [-0.64943288  0.64943288]
[[3.33333333e-01 3.33333333e-01 3.33333333e-01]
 [6.80067182e-01 4.38642543e-04 3.19494128e-01]]
value of hypernet br policy [-0.34549134  0.34549134]
[[3.33333333e-01 3.33333333e-01 3.33333333e-01]
 [7.03098178e-01 4.29598149e-04 2.96472222e-01]]
[[0.         1.         0.        ]
 [0.33333333 0.33333333 0.33333333]]
value of table br policy [ 0.40662596 -0.40662596]
[[0.32554018 0.01904377 0.65541601]
 [0.33333333 0.33333333 0.33333333]]
value of h

  1%|          | 506/100000 [00:08<14:31, 114.19it/s]

...

[[0.33333333 0.33333333 0.33333333]
 [0.61026716 0.18761644 0.20211643]]
[[0.         1.         0.        ]
 [0.33333333 0.33333333 0.33333333]]
value of table br policy [ 0.40815073 -0.40815073]
[[0.67471904 0.21467647 0.1106045 ]
 [0.33333333 0.33333333 0.33333333]]
value of hypernet br policy [ 0.05065671 -0.05065671]
[[0.67131919 0.21740881 0.11127196]
 [0.33333333 0.33333333 0.33333333]]
[[0.33333333 0.33333333 0.33333333]
 [0.         1.         0.        ]]
value of table br policy [-0.56004723  0.56004723]
[[0.33333333 0.33333333 0.33333333]
 [0.61037123 0.1852238  0.20440494]]
value of hypernet br policy [ 0.05383032 -0.05383032]
[[0.33333333 0.33333333 0.33333333]
 [0.60496289 0.18902238 0.20601478]]
[[0.         1.         0.        ]
 [0.33333333 0.33333333 0.33333333]]
value of table br policy [ 0.3989481 -0.3989481]
[[0.67341399 0.21312337 0.11346266]
 [0.33333333 0.33333333 0.33333333]]
value of hypernet br policy [ 0.04927437 -0.04927437]
[[0.67341399 0.21312337 0.1134

 20%|██        | 20007/100000 [02:59<13:24, 99.44it/s] 

[[0.33333333 0.33333333 0.33333333]
 [0.60042876 0.20764203 0.19192919]]
[[0.         1.         0.        ]
 [0.33333333 0.33333333 0.33333333]]
value of table br policy [ 0.40849957 -0.40849957]
[[0.68373919 0.20120779 0.11505305]
 [0.33333333 0.33333333 0.33333333]]
value of hypernet br policy [ 0.0262585 -0.0262585]
[[0.68501079 0.201186   0.11380313]
 [0.33333333 0.33333333 0.33333333]]
[[0.33333333 0.33333333 0.33333333]
 [0.         1.         0.        ]]
value of table br policy [-0.57120766  0.57120766]
[[0.33333333 0.33333333 0.33333333]
 [0.5990057  0.20924628 0.19174796]]
value of hypernet br policy [ 0.02559218 -0.02559218]
[[0.33333333 0.33333333 0.33333333]
 [0.60016334 0.20805824 0.19177841]]
[[0.         1.         0.        ]
 [0.33333333 0.33333333 0.33333333]]
value of table br policy [ 0.40838493 -0.40838493]
[[0.68405205 0.20185252 0.11409547]
 [0.33333333 0.33333333 0.33333333]]
value of hypernet br policy [ 0.02655986 -0.02655986]
[[0.68405205 0.20185252 0.1140

 20%|██        | 20041/100000 [05:05<20:19, 65.57it/s]


KeyboardInterrupt: 

In [ ]:
ind = -1
nn_config
game
input_nets[0]

In [ ]:
# nn_policy = neural_policies.create_policy_net(game, nn_config)
ind = ind + 1
nn_player = 1
nn_policy = input_nets[ind]
nn_tab_policy = neural_policies.nn_to_tabular_policy(game, nn_policy, nn_player)
# br_pol = main.tabular_br(game, 1-nn_player, nn_tab_policy)
print(nn_tab_policy.action_probability_array)

# main.policy_gradient_br(
#     game, nn_config, reinforce_config, nn_player, nn_policy, nn_tab_policy
# )

br = main.compute_best_response_tabular_policy(game, nn_tab_policy, 1-nn_player)
print(br.action_probability_array)
policies = [nn_tab_policy, br] if nn_player == 0 else [br, nn_tab_policy]
combined_policy = main.combine_tabular_policies(game, *policies)
print("value of table br policy", main.policy_value(game, combined_policy))

nn_br = neural_policies.nn_to_tabular_policy(game, policy_nets[1-nn_player], 1-nn_player, nn_policy)
print(nn_br.action_probability_array)
policies = [nn_tab_policy, nn_br] if nn_player == 0 else [nn_br, nn_tab_policy]
combined_policy = main.combine_tabular_policies(game, *policies)
print("value of hypernet br policy", main.policy_value(game, combined_policy))

In [ ]:
pnout = policy_nets[1].model_output(input_nets[ind])
[print(x) for x in pnout.parameters()]

In [ ]:
pnout
game.information_state_tensor_shape()

In [ ]:
g2 = pyspiel.load_game("leduc_poker")
g2.information_state_tensor_shape()
s = game.new_initial_state()
s.information_state_tensor(1)

# test setting weights

In [ ]:
import neural_policies
importlib.reload(neural_policies)
import torch.nn as nn

In [ ]:
mlp = neural_policies.create_policy_net(game, nn_config)

In [ ]:
new_weights = torch.ones_like(mlp.get_weights(), requires_grad=True)

In [ ]:
new_weights.requires_grad, new_weights.grad

In [ ]:
mlp.set_weights(new_weights)

In [ ]:
mlp.network[0].weight.grad

In [ ]:
res = mlp.forward(torch.zeros(1))

In [ ]:
loss = res.sum()
loss.backward()

In [ ]:
mlp.network[0].weight.grad

In [ ]:
new_weights.grad

In [ ]:
linear = nn.Linear(2, 2)
linear.weight.grad

In [ ]:
new_weights = torch.ones(8, requires_grad=True)
# linear.weight = nn.Parameter(new_weights)
linear.weight.requires_grad = False
linear.weight.copy_(new_weights[:4].view(linear.weight.shape))

In [ ]:
inp = torch.ones(2, requires_grad=True)
loss = linear(inp).sum()

In [ ]:
loss.backward()

In [ ]:
inp.grad
linear.weight.grad

In [ ]:
new_weights.grad